<a href="https://colab.research.google.com/github/soumiksinha2004-hash/ml/blob/main/Scorecard_project_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import packages ::

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import numpy as np, pandas as pd
import os
import re

In [ ]:
# Display all columns and values in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Change the directory :::

In [ ]:
os.chdir("/content/drive/MyDrive/Scorecard_project")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Scorecard_project'

In [ ]:
ls

### Load the data :::

In [ ]:
cdf = pd.read_excel("cibil_data_with_target.xlsx")
cdf.head()

In [ ]:
print(cdf.target1.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(cdf.drop('target1', axis=1), cdf['target1'], test_size=0.2, random_state=42)

# Create a 'tag' column in the original DataFrame
cdf['tag'] = 'train'
cdf.loc[X_test.index, 'tag'] = 'test'

# Display the first few rows with the new 'tag' column
display(cdf.head())

# Display the value counts for the 'tag' column to verify the split
display(cdf['tag'].value_counts())

## EDA :::

### Univariate analysis :::

In [ ]:
missing_percentage = cdf[cdf['tag'] == 'train'].isnull().sum() / len(cdf[cdf['tag'] == 'train']) * 100
missing_percentage_df = pd.DataFrame({'Missing Percentage': missing_percentage})

numerical_cols = [i for i in cdf.columns if cdf[i].nunique()>20]
categorical_cols = [i for i in cdf.columns if cdf[i].nunique()<=20]


numerical_stats = cdf[cdf['tag'] == 'train'][numerical_cols].describe(percentiles=[.25, .50, .75, .99]).T[['25%', '50%', '75%', '99%']]
numerical_stats['variance'] = cdf[cdf['tag'] == 'train'][numerical_cols].var()


# Combine the results
combined_stats = missing_percentage_df.join(numerical_stats, how='left')

# Add feature types
combined_stats['Feature Type'] = 'Numerical'
combined_stats.loc[categorical_cols, 'Feature Type'] = 'Categorical'
combined_stats = combined_stats.reset_index()
combined_stats.to_excel("univariate_analysis.xlsx", index=False)
display(combined_stats.sort_values(by='Missing Percentage', ascending=False))

In [ ]:
selected_features = list(combined_stats[combined_stats['Missing Percentage']<71]['index'])

selected_features = [i for i in selected_features if i not in ['num_dbt', 'num_lss']]

In [ ]:
len(selected_features)

In [ ]:
selected_categorical_features = [i for i in selected_features if i in categorical_cols]
selected_categorical_features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in selected_categorical_features:
    plt.figure(figsize=(5, 4))
    ax = sns.countplot(x=col, data=cdf[cdf['tag'] == 'train'], palette='viridis') # Use 'viridis' palette for different colors
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')

    total = len(cdf[col].dropna()) # Calculate total count for percentage
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2
        y = p.get_height()
        ax.annotate(percentage, (x, y), ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

### Bivariate analysis :::

In [ ]:
features_to_remove = ['num_deliq_6mts','num_deliq_6_12mts','num_sub_6mts','num_sub_12mts','num_dbt_6mts','num_dbt_12mts','num_lss_6mts','num_lss_12mts']
selected_features = [i for i in selected_features if i not in features_to_remove]
len(selected_features)

In [ ]:
cdf1 = cdf[selected_features]
cdf1.head()

In [ ]:
!pip install xlsxwriter

In [ ]:
cdf1.GENDER.value_counts()

###### Encoding :::

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder, LabelEncoder

ed_order = ['SSC', '12TH', 'GRADUATE', 'UNDER GRADUATE', 'POST-GRADUATE','PROFESSIONAL', 'OTHERS']

ordencoder = OrdinalEncoder(categories=[ed_order])
lblencoder = LabelEncoder()
stdscaler = StandardScaler()


# cdf2['ED_coded'] = ordencoder.fit_transform(cdf2[['EDUCATION']])
cdf1['last_prod_enq2_coded'] = lblencoder.fit_transform(cdf1['last_prod_enq2'])
cdf1['first_prod_enq2_coded'] = lblencoder.fit_transform(cdf1['first_prod_enq2'])

cdf1['MARITALSTATUS_coded'] = 1
cdf1.loc[cdf1['MARITALSTATUS'] == 'Single', 'MARITALSTATUS_coded'] = 0

cdf1['education_coded'] = ordencoder.fit_transform(cdf1[['EDUCATION']])

cdf1['gender_coded'] = 1
cdf1.loc[cdf1['GENDER'] == 'F', 'gender_coded'] = 0



cdf1 = cdf1.drop(['last_prod_enq2','first_prod_enq2','MARITALSTATUS','EDUCATION','GENDER'], axis=1)


cdf1.head()

#### Correlation analysis :::

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Select only numerical columns for correlation analysis
numerical_cols = cdf1.select_dtypes(include=np.number).columns.tolist()

# Calculate the correlation matrix
correlation_matrix = cdf1[numerical_cols].corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f")
plt.title('Cross-Correlation Heatmap of Numerical Features')
plt.show()

In [ ]:
feat_to_remove = ['max_recent_level_of_deliq','max_deliq_12mts','num_times_30p_dpd','num_std','num_std_6mts',
                  'CC_enq_L6m','CC_enq_L12m','PL_enq_L6m','PL_enq_L12m','enq_L6m','enq_L3m','pct_PL_enq_L6m_of_ever','pct_CC_enq_L6m_of_ever']

filtered_cols = [i for i in cdf1.columns if i not in feat_to_remove]
cdf1 = cdf1[filtered_cols].copy()
cdf1.shape

In [ ]:
cdf1.columns

In [ ]:
import pandas as pd
import numpy as np

def calculate_iv_woe(df, target):
    """
    Calculates the Information Value (IV) and Weight of Evidence (WOE) for all features in a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        target (str): The name of the target column (must be binary: 0 or 1).

    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: A DataFrame containing the IV for each feature.
            - dict: A dictionary where keys are feature names and values are DataFrames containing WOE details.
    """
    iv_values = {}
    woe_reports = {}
    for col in df.columns:
        if col == target:
            continue

        # For numerical features, bin the data
        if df[col].nunique() > 10:
            # Create bins for numerical features. Adjust the number of bins as needed.
            try:
                bins = pd.qcut(df[col], q=10, labels=False, duplicates='drop')
                data = pd.DataFrame({'bins': bins, 'target': df[target]})
                data = data.groupby('bins')['target'].agg(['count', 'sum']).reset_index()
                data.columns = ['bin', 'total', 'good']
                data['bad'] = data['total'] - data['good']
                # data.columns = ['bin', 'total', 'bad']
                # data['good'] = data['total'] - data['bad']
            except (ValueError, KeyError):
                 iv_values[col] = 0 # IV is 0 if binning fails
                 woe_reports[col] = pd.DataFrame() # Empty DataFrame for WOE if binning fails
                 continue

        # For categorical features, use the categories as groups
        else:
            data = df.groupby(col)[target].agg(['count', 'sum']).reset_index()
            data.columns = ['bin', 'total', 'good']
            data['bad'] = data['total'] - data['good']
            # data.columns = ['bin', 'total', 'bad']
            # data['good'] = data['total'] - data['bad']


        # Calculate WOE and IV
        data['bad_rate'] = data['bad'] / data['bad'].sum()
        data['good_rate'] = data['good'] / data['good'].sum()

        # Avoid division by zero or log(0)
        data['woe'] = np.log((data['good_rate'] + 1e-6) / (data['bad_rate'] + 1e-6))
        data['iv'] = (data['good_rate'] - data['bad_rate']) * data['woe']

        iv_values[col] = data['iv'].sum()
        woe_reports[col] = data[['bin', 'total', 'good', 'bad', 'good_rate', 'bad_rate', 'woe', 'iv']]


    iv_df = pd.DataFrame.from_dict(iv_values, orient='index', columns=['IV'])
    iv_df.index.name = 'Feature'
    iv_df = iv_df.sort_values(by='IV', ascending=False).reset_index()

    return iv_df, woe_reports

# Example usage (assuming your DataFrame is cdf and target is 'target1')
iv_report, woe_reports = calculate_iv_woe(cdf1[cdf1['tag']=='train'], 'target1')
display(iv_report)
# To display the WOE report for a specific feature, e.g., 'AGE':
# display(woe_reports['AGE'])

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df):
    """
    Calculates the Variance Inflation Factor (VIF) for numerical features in a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the VIF for each numerical feature.
    """
    numerical_df = df.select_dtypes(include=np.number).fillna(-99999) # Replace NaN with -99999

    vif_data = pd.DataFrame()
    vif_data["feature"] = numerical_df.columns

    vif_data["VIF"] = [variance_inflation_factor(numerical_df.values, i)
                       for i in range(numerical_df.shape[1])]

    return vif_data.sort_values(by='VIF', ascending=False)

# Example usage (assuming your DataFrame is cdf)
vif_report = calculate_vif(cdf1[cdf1['tag']=='train'].drop(['target1'], axis=1))
display(vif_report)

In [ ]:
from scipy.stats import f_oneway
import pandas as pd
import numpy as np

def calculate_anova(df, target):
    """
    Performs ANOVA tests for numerical features against a binary target variable.

    Args:
        df (pd.DataFrame): The input DataFrame.
        target (str): The name of the binary target column (0 or 1).

    Returns:
        pd.DataFrame: A DataFrame containing the F-value and P-value for each numerical feature.
    """
    numerical_cols = [col for col in df.columns if df[col].nunique() > 20]

    anova_results = {}
    for col in numerical_cols:
        if col != target:
            # Handle potential NaN values by dropping them for the ANOVA test
            groups = df.dropna(subset=[col, target]).groupby(target)[col].apply(list)
            if len(groups) > 1: # ANOVA requires at least two groups
                try:
                    fvalue, pvalue = f_oneway(*groups)
                    anova_results[col] = {'anova F-value': fvalue, 'anova P-value': pvalue}
                except ValueError: # Handle cases where groups have no variance
                    anova_results[col] = {'anova F-value': np.nan, 'anova P-value': np.nan}


    anova_report = pd.DataFrame.from_dict(anova_results, orient='index')
    anova_report.index.name = 'Feature'
    return anova_report.sort_values(by='anova P-value').reset_index()

# Example usage (assuming your DataFrame is cdf and target is 'target1')
numerical_cols = [i for i in cdf1.columns if cdf1[i].nunique()>20]+['target1']
anova_report = calculate_anova(cdf1.loc[cdf1['tag']=='train', numerical_cols], 'target1')
display(anova_report)

In [ ]:
import scipy.stats as ss
import pandas as pd
import numpy as np

def cramers_v(x, y):
    """
    Calculates Cramer's V for two categorical variables.

    Args:
        x (pd.Series): The first categorical variable.
        y (pd.Series): The second categorical variable (target variable).

    Returns:
        float: The Cramer's V value.
    """
    confusion_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    # Corrected Cramer's V
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Example usage (assuming your DataFrame is cdf and target is 'target1')
categorical_cols = [i for i in cdf1.columns if cdf1[i].nunique()<=20]+['target1']
cramers_v_results = {col: cramers_v(cdf1[col], cdf1['target1']) for col in categorical_cols if col != 'target1'}
cramers_v_report = pd.DataFrame.from_dict(cramers_v_results, orient='index', columns=["Cramer's V"]).sort_values(by="Cramer's V", ascending=False).reset_index()
display(cramers_v_report)

In [ ]:
bivariate_analysis = iv_report.merge(vif_report, left_on='Feature', right_on='feature', how='left').drop(['feature'], axis=1)
bivariate_analysis = bivariate_analysis.merge(anova_report, left_on='Feature', right_on='Feature', how='left')
bivariate_analysis = bivariate_analysis.merge(cramers_v_report, left_on='Feature', right_on='index', how='left').drop(['index'], axis=1)

bivariate_analysis.head(3)

In [ ]:
bivariate_analysis['type'] = 'numerical'
bivariate_analysis.loc[bivariate_analysis['Feature'].isin(categorical_cols), 'type'] = 'categorical'
bivariate_analysis.to_excel("bivariate_analysis.xlsx", index=False)
bivariate_analysis.head()

In [ ]:
cdf1 = cdf1[l+['target1','tag']]
cdf1.shape

#### Event rate analysis :::

In [ ]:
def export_to_xl(df_name, var_name, writer, iterator=0, pd_var=None):

    VAR_NAME = var_name[:31] if len(var_name) > 31 else var_name

    from xlsxwriter.utility import xl_rowcol_to_cell

    start_row = 16
    start_col = 2 + (10 * iterator)
    endrow = start_row + len(df_name)

    chart_pos_var = xl_rowcol_to_cell(1, 2 + (10 * iterator))

    cat_char_strt = xl_rowcol_to_cell(start_row+1, 3 + (10 * iterator), row_abs=True, col_abs=True)
    cat_char_end = xl_rowcol_to_cell(endrow-1, 3 + (10 * iterator), row_abs=True, col_abs=True)

    bp_nm_char = xl_rowcol_to_cell(start_row, 6 + (10 * iterator), row_abs=True, col_abs=True)
    bp_val_char_strt = xl_rowcol_to_cell(start_row+1, 6 + (10 * iterator), row_abs=True, col_abs=True)
    bp_val_char_end = xl_rowcol_to_cell(endrow-1, 6 + (10 * iterator), row_abs=True, col_abs=True)

    lp_nm_char = xl_rowcol_to_cell(start_row, 7 + (10 * iterator), row_abs=True, col_abs=True)
    lp_val_char_strt = xl_rowcol_to_cell(start_row+1, 7 + (10 * iterator), row_abs=True, col_abs=True)
    lp_val_char_end = xl_rowcol_to_cell(endrow-1, 7 + (10 * iterator), row_abs=True, col_abs=True)

    total_event_rate_strt = xl_rowcol_to_cell(start_row+1, 8 + (10 * iterator), row_abs=True, col_abs=True)
    total_event_rate_end = xl_rowcol_to_cell(endrow-1, 8 + (10 * iterator), row_abs=True, col_abs=True)

    bar_plot_name = "=" + VAR_NAME + "!" + bp_nm_char
    bar_plot_cat = "=" + VAR_NAME + "!" + cat_char_strt + ":" + cat_char_end
    bar_plot_values = "=" + VAR_NAME + "!" + bp_val_char_strt + ":" + bp_val_char_end

    line_plot_name = "=" + VAR_NAME + "!" + lp_nm_char
    line_plot_cat = "=" + VAR_NAME + "!" + cat_char_strt + ":" + cat_char_end
    line_plot_values = "=" + VAR_NAME + "!" + lp_val_char_strt + ":" + lp_val_char_end

    total_event_rate_values = "=" + VAR_NAME + "!" + total_event_rate_strt + ":" + total_event_rate_end

    # Convert the dataframe to an XlsxWriter Excel object.
    df_name.to_excel(writer, sheet_name=VAR_NAME, startcol=start_col, startrow=start_row, index=False)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets[VAR_NAME]

    perc_fmt = workbook.add_format({'num_format': '0.00%', 'align': 'center', 'valign': 'center'})
    num_fmt = workbook.add_format({'num_format': '#,##0', 'align': 'center', 'valign': 'center'})
    # char_fmt = workbook.add_format({'align': 'center', 'valign': 'center'})

    # Create a (primary) column chart.
    bar_plot = workbook.add_chart({'type': 'column'})

    # Configure the data series for the primary chart.
    bar_plot.add_series({
        'name':       bar_plot_name,
        'categories': bar_plot_cat,
        'values':     bar_plot_values,
        'fill': {
            'color': '#C34B45'  # Red Accent 2
        }
    })

    # Create a (secondary) line chart.
    line_plot = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart.
    # y2_axis = secondary Y axis.
    line_plot.add_series({
        'name':       line_plot_name,
        'categories': line_plot_cat,
        'values':     line_plot_values,
        'y2_axis':    True,
        'line': {
            'color': '#FFA500',  # Orange
            'dash_type': 'solid'
        }
    })

    # Configure the data series for the Total_Event_Rate line chart.
    line_plot.add_series({
        'name':       'Overall_Event_Rate',
        'categories': line_plot_cat,
        'values':     total_event_rate_values,
        'y2_axis':    True,
        'line': {
            'color': 'black',
            'dash_type': 'dash'
        }
    })
    # Combining the charts.
    bar_plot.combine(line_plot)

    # Adding chart title.
    if pd_var:
        title = var_name + "_" + pd_var
    else:
        title = var_name + "_OVERALL"

    bar_plot.set_title({'name': title})
    bar_plot.set_y_axis({'major_gridlines': {'visible': False}})
    bar_plot.set_legend({'position': 'bottom'})

    bold = workbook.add_format({'bold': 1})
    worksheet.write('B1', '=HYPERLINK("#Summary!A1","Back")', bold)
    bar_plot.combine(line_plot)

    # Adding chart title.
    if pd_var:
        title = var_name + "_" + pd_var
    else:
        title = var_name + "_OVERALL"

    bar_plot.set_title({'name': title})
    bar_plot.set_y_axis({'major_gridlines': {'visible': False}})
    bar_plot.set_legend({'position': 'bottom'})

    bold = workbook.add_format({'bold': 1})
    worksheet.write('B1', '=HYPERLINK("#Summary!A1","Back")', bold)
    bar_plot.combine(line_plot)

    # Adding chart title.
    if pd_var:
        title = var_name + "_" + pd_var
    else:
        title = var_name + "_OVERALL"

    bar_plot.set_title({'name': title})
    bar_plot.set_y_axis({'major_gridlines': {'visible': False}})
    bar_plot.set_legend({'position': 'bottom'})

    bold = workbook.add_format({'bold': 1})
    worksheet.write('B1', '=HYPERLINK("#Summary!A1","Back")', bold)

    range_strt = xl_rowcol_to_cell(start_row, 2 + (10 * iterator))
    range_end = xl_rowcol_to_cell(endrow, 10 + (10 * iterator))
    range_var = range_strt + ":" + range_end

    num_range = re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 4 + (10 * iterator))) + ":" + re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 5 + (10 * iterator)))
    perc_range = re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 6 + (10 * iterator))) + ":" + re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 7 + (10 * iterator)))
    perc_range2 = re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 8 + (10 * iterator))) + ":" + re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 9 + (10 * iterator)))
    num_range2 = re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 9 + (10 * iterator))) + ":" + re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 10 + (10 * iterator)))
    num_range3 = re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 10 + (10 * iterator))) + ":" + re.sub('[0-9]', '', xl_rowcol_to_cell(start_row, 11 + (10 * iterator)))

    border_format = workbook.add_format({
        'border': 1,
        'align': 'center',
        'font_size': 10
    })

    worksheet.conditional_format(range_var, {'type': 'no_blanks', 'format': border_format})

    # Insert the chart into the worksheet
    worksheet.insert_chart(chart_pos_var, bar_plot)
    worksheet.hide_gridlines(2)
    worksheet.set_column(num_range, 10.25, num_fmt)
    worksheet.set_column(perc_range, 10.25, perc_fmt)
    worksheet.set_column(perc_range2, 10.25, perc_fmt)
    worksheet.set_column(num_range2, 10.25, num_fmt)
    worksheet.set_column(num_range3, 10.25, num_fmt)
    worksheet.set_zoom(80)

    # writer.save()  # saving is done outside of this loop


In [ ]:
# Function for creating Character Variable Tables
def char_var_tbl(df_chk_chars_concat, dep_var, i, writer1):
    df_v1_EV = df_chk_chars_concat.groupby([i])[[dep_var, 'DF_RECS_CNT']].sum().reset_index()
    df_v1_EV['POP_RATE'] = df_v1_EV['DF_RECS_CNT']/df_v1_EV['DF_RECS_CNT'].sum()
    df_v1_EV['Event_Rate'] = df_v1_EV[dep_var]/df_v1_EV['DF_RECS_CNT']
    df_v1_EV['Total_Event_Rate'] = df_v1_EV[dep_var].sum()/df_v1_EV['DF_RECS_CNT'].sum()
    df_v1_EV['Variable'] = i
    df_v1_EV.rename(columns={i: 'ATTRIBUTE'}, inplace=True)
    df_v1_EV['MIN_VALUE'] = "NA"
    df_v1_EV['MAX_VALUE'] = "NA"

    # CALCULATING VALUES FOR GROUP TOTALS
    columns = ['Variable', 'ATTRIBUTE', dep_var, 'DF_RECS_CNT', 'POP_RATE',
              'Event_Rate', 'Total_Event_Rate', 'MAX_VALUE', 'MIN_VALUE']
    data = [[i, "Total", df_v1_EV[dep_var].sum(), df_v1_EV['DF_RECS_CNT'].sum(),
            df_v1_EV['POP_RATE'].sum(), df_v1_EV[dep_var].sum()/df_v1_EV['DF_RECS_CNT'].sum(),
            "NA", "NA", "NA"]]

    df_v1_EV_tot = pd.DataFrame(data, columns=columns)

    DF_CHAR = pd.concat([df_v1_EV, df_v1_EV_tot], axis=0, ignore_index=True)
    DF_CHAR = DF_CHAR[['Variable', 'ATTRIBUTE', dep_var, 'DF_RECS_CNT', 'POP_RATE',
                      'Event_Rate', 'Total_Event_Rate', 'MIN_VALUE', 'MAX_VALUE']]
    return DF_CHAR


In [ ]:
def num_var_tbl(df_chk_nums_concat, dep_var, i, writer1):
    rank_var = i + "_rank"
    bins = pd.qcut(df_chk_nums_concat[i], 10, duplicates="drop")

    def format_interval(interval):
        lower = interval.left
        higher = interval.right

        if lower >= pow(10, 6):
            lower = f"{str(round(lower / pow(10, 6), 1))}M"
        elif lower >= pow(10, 3):
            lower = f"{str(round(lower / pow(10, 3), 1))}K"
        else:
            lower = str(round(lower, 1))

        if higher >= pow(10, 6):
            higher = f"{str(round(higher / pow(10, 6), 1))}M"
        elif higher >= pow(10, 3):
            higher = f"{str(round(higher / pow(10, 3), 1))}K"
        else:
            higher = str(round(higher, 1))

        return f"{lower} - {higher}"

    bins = bins.apply(format_interval)
    df_chk_nums_concat[rank_var] = bins

    df_v1_num_EV = df_chk_nums_concat.groupby(rank_var)[[dep_var, 'DF_RECS_CNT']].sum().reset_index()
    df_v1_num_EV['POP_RATE'] = df_v1_num_EV['DF_RECS_CNT'] / df_v1_num_EV['DF_RECS_CNT'].sum()

    df_v1_num_MIN_MAX = df_chk_nums_concat.groupby(rank_var).agg({i: ['min', 'max']}).reset_index()
    df_v1_num_MIN_MAX.columns = df_v1_num_MIN_MAX.columns.droplevel(0)
    df_v1_num_MIN_MAX.columns = [rank_var, 'MIN_VALUE', 'MAX_VALUE']

    df_v1_num_CMP2 = pd.merge(left=df_v1_num_EV, right=df_v1_num_MIN_MAX, how='left', on=rank_var)
    df_v1_num_CMP2['Event_Rate'] = df_v1_num_CMP2[dep_var] / df_v1_num_CMP2['DF_RECS_CNT']
    df_v1_num_CMP2['Total_Event_Rate'] = df_v1_num_CMP2[dep_var].sum() / df_v1_num_CMP2['DF_RECS_CNT'].sum()
    df_v1_num_CMP2['Variable'] = i
    df_v1_num_CMP2.rename(columns={rank_var: 'ATTRIBUTE'}, inplace=True)

    # CALCULATING VALUES FOR GROUP TOTALS
    columns = ['Variable', 'ATTRIBUTE', dep_var, 'DF_RECS_CNT', 'POP_RATE',
               'Event_Rate', 'Total_Event_Rate', 'MAX_VALUE', 'MIN_VALUE']
    data = [[i, 'Total', df_v1_num_CMP2[dep_var].sum(), df_v1_num_CMP2['DF_RECS_CNT'].sum(),df_v1_num_CMP2['POP_RATE'].sum(),
             df_v1_num_CMP2[dep_var].sum() / df_v1_num_CMP2['DF_RECS_CNT'].sum(),
             "NA", "NA", "NA"]]

    df_v1_num_CMP2_tot = pd.DataFrame(data, columns=columns)
    # df_v1_num_CMP2['Total_Event_Rate'] = [df_v1_num_CMP2_tot['Total_Event_Rate']]*len(df_v1_num_CMP2)

    DF_NUM = pd.concat([df_v1_num_CMP2, df_v1_num_CMP2_tot], axis=0, ignore_index=True)
    DF_NUM = DF_NUM[['Variable', 'ATTRIBUTE', dep_var, 'DF_RECS_CNT',
                     'POP_RATE', 'Event_Rate', 'Total_Event_Rate', 'MIN_VALUE', 'MAX_VALUE']]

    return DF_NUM

In [ ]:
# Main function
def bivariate(df_chk,depvar,output_file_nm,MODEL_PERIOD_FLAG=None):
    from xlsxwriter.utility import xl_rowcol_to_cell

    numerical_cols = [i for i in df_chk.columns if df_chk[i].nunique()>20]
    if depvar not in numerical_cols:
        numerical_cols.append(depvar)

    categorical_cols = [i for i in df_chk.columns if df_chk[i].nunique()<=20]
    if depvar in categorical_cols:
        categorical_cols.remove(depvar)

    df_chk_chars = df_chk[categorical_cols]
    df_chk_nums = df_chk[numerical_cols]

    # df_chk_chars = df_chk.select_dtypes(include = object)
    # df_chk_nums = df_chk.select_dtypes(include = 'number')

    print("all features:",df_chk.columns)
    print("number type features:",df_chk_nums.columns)
    print("character type features:",df_chk_chars.columns)


    max_cat = 10000
    df_chk_depvar = df_chk[[depvar]]
    if MODEL_PERIOD_FLAG :
        df_chk_pdvar = df_chk[[MODEL_PERIOD_FLAG]]
    else :
        df_chk_pdvar = pd.DataFrame()
    df_chk_pd_depvar = pd.concat([df_chk_depvar, df_chk_pdvar], axis=1)
    try:
        df_chk_nums.drop([depvar],axis=1,inplace=True)
        # df_chk_chars.drop([depvar],axis=1,inplace=True)
        pass
    except ValueError:
        pass

    varlist_num = df_chk_nums.columns
    varlist_char = df_chk_chars.columns

    varlist_comb = df_chk_chars.columns.tolist()
    for i1 in varlist_comb:
        if df_chk_chars[i1].unique().shape[0]>=max_cat :
            varlist_comb.remove(i1)

    varlist_comb.extend(df_chk_nums.columns.tolist())
    varlist_comb2 = l =  [None] * len(varlist_comb)

    for i in np.arange(len(varlist_comb)):
        link_name = varlist_comb[i][:31] if len(varlist_comb[i])>31 else varlist_comb[i]
        varlist_comb2[i] = '=HYPERLINK("#'+link_name+'!A1","'+link_name+'")'

    summ_tab_df = pd.DataFrame(list(zip(varlist_comb, varlist_comb2)),
                columns=['Variable Name','Variable Sheet Link'])
    #summ_tab_df = pd.DataFrame(varlist_comb,columns=["Variable Name"])
    #print(summ_tab_df["Variable Name"])

    df_chk_chars_concat = pd.concat([df_chk_chars, df_chk_depvar], axis=1)
    df_chk_nums_concat = pd.concat([df_chk_nums, df_chk_pd_depvar], axis=1)


    df_chk_chars_concat['DF_RECS_CNT'] = 1
    df_chk_nums_concat['DF_RECS_CNT'] = 1

    df_chk_chars_concat.fillna("BLANK", inplace=True)
    df_chk_nums_concat.fillna(-999999, inplace=True)


    writer1 = pd.ExcelWriter(output_file_nm, engine='xlsxwriter')

    summ_tab_df.to_excel(writer1, sheet_name="Summary", startcol=2 , startrow=5 , index=False)
    wbook = writer1.book
    wsheet = writer1.sheets["Summary"]
    wsheet.set_column(2, 3, 25)
    summ_range_strt = xl_rowcol_to_cell(5, 2)
    summ_range_end = xl_rowcol_to_cell(5+len(summ_tab_df), 2)
    b_range_var = summ_range_strt+':'+summ_range_end
    link_range_strt = xl_rowcol_to_cell(5, 3)
    link_range_end = xl_rowcol_to_cell(5+len(summ_tab_df), 3)
    link_range_var = link_range_strt+':'+link_range_end
    b_format=wbook.add_format({
                'border':1,
                'align':'center',
                'font_size':10
    })

    link_format=wbook.add_format({'border':1,
                'align':'center',
                'font_size':9
    })
    link_format.set_font_color('blue')

    wsheet.conditional_format(b_range_var, { 'type' : 'no_blanks' , 'format' : b_format } )
    wsheet.conditional_format(link_range_var, { 'type' : 'no_blanks' , 'format' : link_format } )
    wsheet.hide_gridlines(2)
    wsheet.set_zoom(80)


    for i in varlist_char:
        if i != depvar and df_chk_chars_concat[i].unique().shape[0]<max_cat:
            DF_CHAR = char_var_tbl(df_chk_chars_concat,depvar,i,writer1)
            # DF_CHAR = DF_CHAR.sort_values(["DF_RECS_CNT"], ascending=False)
            export_to_xl(DF_CHAR,var_name=i,writer=writer1)

    for i in varlist_num :
        if i != depvar:
            DF_NUM = num_var_tbl(df_chk_nums_concat,depvar,i,writer1)
            export_to_xl(DF_NUM,var_name=i,writer=writer1)

    if MODEL_PERIOD_FLAG:
        pd_tag_array = df_chk[MODEL_PERIOD_FLAG].unique()[:5]
        pd_char_iter = 1

        for j in pd_tag_array :
            df_chars_pd_filt = df_chk_chars_concat[df_chk_chars_concat[MODEL_PERIOD_FLAG]==j]
            for i2 in varlist_char:
                print(i2)
                if i2 != depvar and df_chars_pd_filt[i2].unique().shape[0]<max_cat :
                    DF_CHAR = char_var_tbl(df_chars_pd_filt,depvar,i2,writer1)
                    # DF_CHAR = DF_CHAR.sort_values("DF_RECS_CNT", ascending=False)
                    export_to_xl(DF_CHAR,var_name=i2,writer=writer1,iterator=pd_char_iter,pd_var=j)

            df_num_pd_filt = df_chk_nums_concat[df_chk_nums_concat[MODEL_PERIOD_FLAG]==j]
            for i2 in varlist_num:
                print(i2)
                if i2 != depvar :
                    DF_NUM = num_var_tbl(df_num_pd_filt,depvar,i2,writer1)
                    export_to_xl(DF_NUM,var_name=i2,writer=writer1,iterator=pd_char_iter,pd_var=j)
            pd_char_iter = pd_char_iter+1
        writer1.close()
    else :
        writer1.close()
    print("Done!")

In [ ]:
# Import the input data set for which bivariate analysis is to be done
input_df = cdf1  # use encoding='unicode_escape' in case of unicode utf-8 error

# Define the 'Dependent'/'Event' variable below:
depvar = 'target1'

#[OPTIONAL] : define the 'Period Variable ( E.g. variable with values "01_TRAIN"; "02_VALIDATE", "03_OOT") below ;
Model_flag = 'tag'

#output file name below:
output_file_nm = r"cdf_Bivariate_report_tagged.xlsx"

# [Function Call] !Remove Model_flag from arguments in function below, if it is not present in data/ not defined above
bivariate(input_df,depvar,output_file_nm,MODEL_PERIOD_FLAG=Model_flag)
# bivariate(input_df,depvar,output_file_nm)


In [ ]:
cdf1['education_coded_in234'] = np.where(cdf1['education_coded'].isin([2,3,4]), 1, 0)
cdf1.head()

In [ ]:
cdf1 = cdf1.drop(['education_coded','NETMONTHLYINCOME'], axis=1)
cdf1.shape

In [ ]:
cdf1.columns

### Data preprocessing :::




In [ ]:
cdf2 = cdf1.copy()
cdf2.head()

#### train-test split :::

In [ ]:
# Separate train and test data
cdf2_train = cdf2[cdf2['tag'] == 'train'].copy()
cdf2_test = cdf2[cdf2['tag'] == 'test'].copy()

In [ ]:
# Calculate event rate for training set
event_rate_train = cdf2_train['target1'].mean()

# Calculate event rate for test set
event_rate_test = cdf2_test['target1'].mean()

print(f"Event Rate in Training Set: {event_rate_train:.4f}")
print(f"Event Rate in Test Set: {event_rate_test:.4f}")

#### Outlier treatment - clip with 0.01th and 99th percentile values :::

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
import math

# Identify numerical features based on the number of unique values
numerical_cols = [col for col in cdf2.columns if cdf2[col].nunique() > 20]

# Calculate the number of rows needed for a 3-column layout
n_cols = 3
n_rows = math.ceil(len(numerical_cols) / n_cols)

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# Plot boxplots for each numerical feature
for i, col in enumerate(numerical_cols):
    sns.boxplot(y=cdf2[col], ax=axes[i])  # Changed x to y
    axes[i].set_title(f'Boxplot of {col}')
    axes[i].set_ylabel(col) # Changed xlabel to ylabel

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# Identify numerical features based on the number of unique values
numerical_cols = [col for col in cdf2_train.columns if cdf2_train[col].nunique() > 20]

# Calculate 1st and 99th percentiles for each numerical feature in the training set
p01 = cdf2_train[numerical_cols].quantile(0.01)
p99 = cdf2_train[numerical_cols].quantile(0.99)

# Clip the values in both train and test sets
cdf2_train[numerical_cols] = cdf2_train[numerical_cols].clip(lower=p01, upper=p99, axis=1)
cdf2_test[numerical_cols] = cdf2_test[numerical_cols].clip(lower=p01, upper=p99, axis=1)

print("Train set after clipping:")
display(cdf2_train.head())

print("\nTest set after clipping:")
display(cdf2_test.head())

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
import math

# Identify numerical features based on the number of unique values
numerical_cols = [col for col in cdf2.columns if cdf2[col].nunique() > 20]

# Calculate the number of rows needed for a 3-column layout
n_cols = 3
n_rows = math.ceil(len(numerical_cols) / n_cols)

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# Plot boxplots for each numerical feature
for i, col in enumerate(numerical_cols):
    sns.boxplot(y=cdf2_train[col], ax=axes[i])  # Changed x to y
    axes[i].set_title(f'Boxplot of {col}')
    axes[i].set_ylabel(col) # Changed xlabel to ylabel

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

#### Missing value imputation ::::

In [ ]:
features = [i for i in cdf2_train.columns if i not in ['tag','target1']]
print(features)

# Calculate median for each numerical feature in the training set
medians = cdf2_train[features].median()

# Fill missing values in both train and test sets with the calculated medians
cdf2_train[features] = cdf2_train[features].fillna(medians)
cdf2_test[features] = cdf2_test[features].fillna(medians)

print("Missing values in train set after imputation:")
print(cdf2_train[features].isnull().sum())

print("\nMissing values in test set after imputation:")
print(cdf2_test[features].isnull().sum())

#### Scalling : standard scalling :::

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import math

# Identify all features
all_cols = cdf2.drop(['tag','target1'], axis=1).columns

# Calculate the number of rows needed for a 3-column layout
n_cols = 3
n_rows = math.ceil(len(all_cols) / n_cols)

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# Plot density plots for each feature
for i, col in enumerate(all_cols):
    sns.kdeplot(x=cdf2_train[col], ax=axes[i])
    axes[i].set_title(f'Density Plot of {col}')
    axes[i].set_xlabel(col)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder, LabelEncoder
import joblib

stdscaler = StandardScaler()

# Fit and transform the scaler on the training data
cdf2_train[features] = stdscaler.fit_transform(cdf2_train[features])

# Save the scaler
joblib.dump(stdscaler, 'std_scaler.pkl')

# Load the scaler and transform the test data
loaded_scaler = joblib.load('std_scaler.pkl')
cdf2_test[features] = loaded_scaler.transform(cdf2_test[features])

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
import math

# Identify all features
all_cols = cdf2.drop(['tag','target1'], axis=1).columns

# Calculate the number of rows needed for a 3-column layout
n_cols = 3
n_rows = math.ceil(len(all_cols) / n_cols)

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

# Plot density plots for each feature
for i, col in enumerate(all_cols):
    sns.kdeplot(x=cdf2_train[col], ax=axes[i])
    axes[i].set_title(f'Density Plot of {col}')
    axes[i].set_xlabel(col)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
cdf2_train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import statsmodels.api as sm

# Add a constant (intercept) to the features
X_train_sm = sm.add_constant(cdf2_train[features].drop(['PL_Flag','time_since_recent_payment','HL_Flag','num_times_60p_dpd'], axis=1))

# Fit the logistic regression model using statsmodels
logit_model = sm.Logit(cdf2_train['target1'], X_train_sm)
result = logit_model.fit()

# Display the summary which includes coefficients and p-values
print(result.summary())

# # Define the models
# lr_model = LogisticRegression(random_state=42)

# Example hyperparameters for tree-based models
dt_model = DecisionTreeClassifier(random_state=42, max_depth=4, min_samples_split=5)
rf_model = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=4, min_samples_leaf=5)
xgb_model = XGBClassifier(random_state=42, n_estimators=50, learning_rate=0.1, max_depth=4)


# Train the models
# lr_model.fit(cdf2_train[features], cdf2_train['target1'])
dt_model.fit(cdf2_train[features], cdf2_train['target1'])
rf_model.fit(cdf2_train[features], cdf2_train['target1'])
xgb_model.fit(cdf2_train[features], cdf2_train['target1'])

print("Models trained successfully.")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import statsmodels.api as sm # Import statsmodels

models = {
    "Logistic Regression": result, # Use the statsmodels result object
    "Decision Tree": dt_model,
    "Random Forest": rf_model,
    "XGBoost": xgb_model
}

train_results = []
test_results = []

# Add a constant to the feature data for statsmodels prediction
X_train_sm = sm.add_constant(cdf2_train[features].drop(['PL_Flag','time_since_recent_payment','HL_Flag','num_times_60p_dpd'], axis=1)) # Corrected feature list
X_test_sm = sm.add_constant(cdf2_test[features].drop(['PL_Flag','time_since_recent_payment','HL_Flag','num_times_60p_dpd'], axis=1)) # Corrected feature list


for name, model in models.items():
    # Use appropriate prediction method based on the model type
    if name == "Logistic Regression":
        y_pred_train = (model.predict(X_train_sm) > 0.5).astype(int) # Predict probabilities and convert to binary
        y_proba_train = model.predict(X_train_sm) # statsmodels predict directly gives probabilities

        y_pred_test = (model.predict(X_test_sm) > 0.5).astype(int)
        y_proba_test = model.predict(X_test_sm)

    else:
        y_pred_train = model.predict(cdf2_train[features]) # Corrected feature list
        y_proba_train = model.predict_proba(cdf2_train[features])[:, 1] # Corrected feature list

        y_pred_test = model.predict(cdf2_test[features]) # Corrected feature list
        y_proba_test = model.predict_proba(cdf2_test[features])[:, 1] # Corrected feature list


    precision_train = precision_score(cdf2_train['target1'], y_pred_train)
    recall_train = recall_score(cdf2_train['target1'], y_pred_train)
    f1_train = f1_score(cdf2_train['target1'], y_pred_train)
    auc_train = roc_auc_score(cdf2_train['target1'], y_proba_train)
    tn_train, fp_train, fn_train, tp_train = confusion_matrix(cdf2_train['target1'], y_pred_train).ravel()


    train_results.append({
        "Model": name,
        "Precision": precision_train,
        "Recall": recall_train,
        "F1 Score": f1_train,
        "AUC": auc_train,
        "True Positives": tp_train,
        "False Positives": fp_train,
        "True Negatives": tn_train,
        "False Negatives": fn_train
    })

    precision_test = precision_score(cdf2_test['target1'], y_pred_test)
    recall_test = recall_score(cdf2_test['target1'], y_pred_test)
    f1_test = f1_score(cdf2_test['target1'], y_pred_test)
    auc_test = roc_auc_score(cdf2_test['target1'], y_proba_test)
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(cdf2_test['target1'], y_pred_test).ravel()

    test_results.append({
        "Model": name,
        "Precision": precision_test,
        "Recall": recall_test,
        "F1 Score": f1_test,
        "AUC": auc_test,
        "True Positives": tp_test,
        "False Positives": fp_test,
        "True Negatives": tn_test,
        "False Negatives": fn_test
    })

train_results_df = pd.DataFrame(train_results)
test_results_df = pd.DataFrame(test_results)

print("Model Evaluation Metrics (Training Data):")
display(train_results_df)

print("\nModel Evaluation Metrics (Test Data):")
display(test_results_df)

In [ ]:
import numpy as np
import pandas as pd # Import pandas if not already imported

def create_decile_report(y_true, y_proba, num_deciles=10):
    """
    Generates a decile-wise report for a binary classification model.

    Args:
        y_true (pd.Series): True labels (0 or 1).
        y_proba (pd.Series): Predicted probabilities for the positive class.
        num_deciles (int): The number of deciles to create.

    Returns:
        pd.DataFrame: A DataFrame containing the decile report.
    """
    # Create a DataFrame with true labels and predicted probabilities
    df = pd.DataFrame({'true_label': y_true, 'predicted_proba': y_proba})

    # Sort by predicted probability in descending order
    df = df.sort_values(by='predicted_proba', ascending=False).reset_index(drop=True)

    # Create deciles based on equal number of instances
    df['decile'] = pd.qcut(df.index, q=num_deciles, labels=False, duplicates='drop') + 1

    # Calculate metrics for each decile
    decile_report = df.groupby('decile').agg(
        population=('decile', 'count'),
        events=('true_label', 'sum')
    ).reset_index()

    decile_report['non_events'] = decile_report['population'] - decile_report['events']
    decile_report['event_rate'] = decile_report['events'] / decile_report['population']

    # Cumulative calculations
    decile_report['cumulative_population'] = decile_report['population'].cumsum()
    decile_report['cumulative_events'] = decile_report['events'].cumsum()
    decile_report['cumulative_non_events'] = decile_report['non_events'].cumsum()

    # Calculate total events and non-events
    total_events = decile_report['events'].sum()
    total_non_events = decile_report['non_events'].sum()

    # Calculate metrics
    decile_report['event_capture_rate'] = decile_report['cumulative_events'] / total_events
    decile_report['non_event_capture_rate'] = decile_report['cumulative_non_events'] / total_non_events
    decile_report['lift'] = decile_report['event_capture_rate'] / (decile_report['cumulative_population'] / decile_report['population'].sum())
    decile_report['gain'] = decile_report['event_capture_rate'] * 100
    decile_report['precision'] = decile_report['cumulative_events'] / decile_report['cumulative_population']
    decile_report['recall'] = decile_report['event_capture_rate']
    decile_report['KS'] = decile_report['event_capture_rate'] - decile_report['non_event_capture_rate']


    return decile_report[['decile', 'population', 'events', 'event_rate', 'lift', 'gain', 'precision', 'recall', 'KS']]

# Define the models (Assuming result, dt_model, rf_model, xgb_model are already trained)
models = {
    "Logistic Regression": result,
    "Decision Tree": dt_model,
    "Random Forest": rf_model,
    "XGBoost": xgb_model
}

for name, model in models.items():
    print(f"Decile Report for {name} (Training Data):")
    # Get predicted probabilities for the training set
    if name == "Logistic Regression":
        # For statsmodels, predict() directly gives probabilities
        y_proba_train = model.predict(sm.add_constant(cdf2_train[features].drop(['PL_Flag','time_since_recent_payment','HL_Flag','num_times_60p_dpd'], axis=1)))
    else:
        y_proba_train = model.predict_proba(cdf2_train[features])[:, 1]

    # Create the decile report for the training data
    decile_report_train = create_decile_report(cdf2_train['target1'], y_proba_train)

    # Display only the first 3 deciles
    display(decile_report_train.head(3))
    print("\n")

In [ ]:
for name, model in models.items():
    print(f"Decile Report for {name} (Test Data):")
    # Get predicted probabilities for the training set
    if name == "Logistic Regression":
        # For statsmodels, predict() directly gives probabilities
        y_proba_test = model.predict(sm.add_constant(cdf2_test[features].drop(['PL_Flag','time_since_recent_payment','HL_Flag','num_times_60p_dpd'], axis=1)))
    else:
        y_proba_test = model.predict_proba(cdf2_test[features])[:, 1]

    # Create the decile report for the training data
    decile_report_test = create_decile_report(cdf2_test['target1'], y_proba_test)

    # Display only the first 3 deciles
    display(decile_report_test.head(3))
    print("\n")